In [171]:
import os
import json
import pandas as pd

In [172]:
def extract_name(data):
    return data['givenName'] + ' ' + data['familyName']

def extract_bio(data):
    return data['profileText']

def get_congresses(data):
    jobs = data['jobPositions']
    for job in jobs:
        try:
            yield job['congressAffiliation']['congress']['congressNumber']
        except:
            print('bad field')

def load_bio(file):
    with open(file,'r') as in_file:
        data = json.load(in_file)
        congresses = list(get_congresses(data))
        return pd.DataFrame({
            'name':[extract_name(data)],
            'bio':[data['profileText']],
            'alive':['deathDate' not in data],
            'recent':[118 in congresses or 119 in congresses]
        })

def df_to_md(df,out_name):
    with open(out_name,'w') as out_file:
        for i in range(0,len(df['name'])):
            out_file.write(f"# {df['name'][i]}\n")
            out_file.write(" \n")
            bio_lines = pd.Series(df['bio'][i].split(';')).apply(lambda x: x.strip())
            for line in bio_lines:
                if 'College' in line or 'University' in line:
                    out_file.write(f"- **{line}**\n")
                else:
                    out_file.write(f"- {line}\n")
            out_file.write(" \n")

In [173]:
files = ['../data/raw/' + file for file in os.listdir('../data/raw')]

In [174]:
df = pd.concat([load_bio(file) for file in files])
df = df.sort_values('name').reset_index(drop=True)

bad field
bad field
bad field
bad field
bad field
bad field
bad field


In [175]:
df_to_md(df,'../full.md')

In [176]:
alive_df = df[df['alive']].reset_index(drop=True)
df_to_md(alive_df,'../recent.md')

In [177]:
recent_df = df[df['alive']].reset_index(drop=True)
df_to_md(alive_df,'../alive.md')